#### Qlunc inputs tutorial by Francisco Costa
# Qlunc (Quantification lidar uncertainties) 
Toolbox to assess uncertainties related with lidar hardware and lidar data processing
In this tutorial you will get 

**Here is missing the first part...when downloading the repository and the first steps, basic CPU requirements**

Importing some packages is always needed:

In [5]:
import pandas as pd
import sys,inspect
from functools import reduce
from operator import getitem
import yaml

First step is to fill the yaml file. We can find it in _Qlunc-V0.9/TestFiles_Qlunc/Working_example_yaml_inputs_file.yml_.
You can open it with your preferred text editor and fill up the lidar parameters. Once all fields are filled, python offers a way to read data from a yaml file.

In [11]:
#Obtain data from _.yaml_ file:

with open (r'../GitHub_Qlunc/TestFiles_Qlunc/Working_example_yaml_inputs_file.yml','r') as file:
    Qlunc_yaml_inputs={}
    docs = yaml.load_all(file, Loader=yaml.FullLoader)
    for doc in docs:      
        for k, v in doc.items():     
            Qlunc_yaml_inputs.setdefault(k,v)

We obtain a dictionary, a well-known object in python, with all the data introduced in the yaml file

In [12]:
Qlunc_yaml_inputs

{'Main_directory': '../GitHub_Qlunc/Main',
 'Components': {'Scanner': {'Name': 'Scanner_Yaml',
   'Origin': [0, 0, 0],
   'Sample rate': 0,
   'Focus distance': [68],
   'Cone angle': [30],
   'Azimuth': [0, 360, 15],
   'stdv focus distance': 1,
   'stdv Cone angle': 0.6,
   'stdv Azimuth': 0.8,
   'Uncertainty function': 'uopc.UQ_Scanner'},
  'Optical Circulator': {'Name': 'OC_Yaml',
   'Insertion loss': 2.1,
   'Uncertainty function': 'uopc.UQ_OpticalCirculator'},
  'Optical Amplifier': {'Name': 'OA_Yaml',
   'Optical amplifier noise figure': '../metadata/NoiseFigure.csv',
   'Optical amplifier gain': 30,
   'Uncertainty function': 'uphc.UQ_Optical_amplifier'},
  'Photodetector': {'Name': 'Photodetector_YAML',
   'Photodetector BandWidth': 380000000.0,
   'Load resistor': 50,
   'Photodetector efficiency': 0.85,
   'Dark current': 5e-09,
   'Photodetector signalP': 0.001,
   'Power interval': [0, 1000, 0.001],
   'Gain TIA': 5000.0,
   'V Noise TIA': 0.00016,
   'Uncertainty functio

Now, we want to create a virtual-twin lidar device by using python object oriented programming features. For that a for each, component, module and even lidar itself is created. We will instantiate those classes once we've created them, so:

In [13]:
exec(open(Qlunc_yaml_inputs['Main_directory']+'/Qlunc_Classes.py').read())   # Execute Qlunc_Classes.py (creating classes for lidar 'objects')

FileNotFoundError: [WinError 2] The system cannot find the file specified: '../Utils '

In [14]:
Qlunc_yaml_inputs['Main_directory']

'../GitHub_Qlunc/Main'

In [16]:
exec(open(Qlunc_yaml_inputs['Main_directory']+'/Qlunc_Classes.py').read())

FileNotFoundError: [WinError 2] The system cannot find the file specified: '../Utils '